In [1]:
#importing used packages
import csv
from selenium import webdriver 
import pandas as pd
import numpy as np
import time
import re

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

## Function to get all products link with addition argument as number of pages want to scrap

In [3]:
def get_product_links(driver,number_of_pages,product_name):
    
    
    product_name = re.subn(" ","+",product_name)[0]  #link has no spaces so we are changing sapce with "+"     
    links=[]
    i=1
    
    #extracting product links from different webpages
    while(i<number_of_pages+1):
        time.sleep(0.5)
        
        #opening different webpages by changing product name and page number for link extraction
        url = "https://www.amazon.in/s?k={}&page=".format(product_name.lower())+str(i)+"&qid=1634839581&ref=sr_pg_1"
        driver.get(url) 
        time.sleep(0.5)
        
        #extracting all the link attached to products name
        all_a = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
        for a in all_a:
            links.append(a.get_attribute("href"))
        i+=1
        
    return links


## Function to get all features of the first product

In [4]:
def get_features_from_first_product(driver,link):
    driver.get(link)
    col = ['Name','Price']
    for x in range(1,100):
        try:
            #extracting all the feactures(values in dark column)
            c = driver.find_element_by_xpath('//*[@id="productDetails_techSpec_section_1"]/tbody/tr['+ str(x) +']/th').text
            col.append(c)
        except:
            break
    return col

## Function to get data from amazon

In [5]:
def get_data_from_amazon(page_number):
    
    #get Product name
    product_name = input("Enter the product name: ")
    
    driver = webdriver.Chrome(PATH) #open browser
    
    link = get_product_links(driver,page_number,product_name) #extracting all products link
    print(len(link))
    a = np.arange(len(link))
    
    col = get_features_from_first_product(driver,link[0]) #extracting features
    product = pd.DataFrame(index = a,columns=col)
    print(col)
    
    #extracting information from all links
    for i in a:
            driver.get(link[i])
            time.sleep(0.5)
            
            #extracting product name
            try:
                name = driver.find_element_by_xpath('//*[@id="productTitle"]').text 
                product.loc[i,'Name'] = name
            except:
                None
                
            #extracting product price
            try:
                price = driver.find_element_by_xpath('//*[@id="priceblock_dealprice"]').text 
                product.loc[i,'Price'] = price
            except:
                None
                
            try:
                #extracting all the features of the current product on the webpage
                c = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
                
                #extracting all the values corresponding to the features 
                d = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//td[@class="a-size-base prodDetAttrValue"]')
                
                #saving the values of the selected featues in the database 
                for x in range(len(c)):
                    if c[x].text in col:
                        product.loc[i,c[x].text] = d[x].text
            except:
                print(link[i])

    driver.quit()
    product.to_csv(product_name+'.csv')
    return product

In [6]:
product = get_data_from_amazon(20) #passing page number for easy test

Enter the product name: wifi router
26
['Name', 'Price', 'Brand', 'Manufacturer', 'Model', 'Package Dimensions', 'Item model number', 'Operating System', 'Mounting Hardware', 'Number Of Items', 'Batteries Required', 'Wireless Type', 'Number of Ports', 'Connector Type', 'Manufacturer', 'Country of Origin', 'Item Weight']


In [7]:
product

,Name,Price,Brand,Manufacturer,Model,Package Dimensions,Item model number,Operating System,Mounting Hardware,Number Of Items,Batteries Required,Wireless Type,Number of Ports,Connector Type,Manufacturer,Country of Origin,Item Weight
0,Cisco RV160W VPN Router | 4 Wireless Ports | W...,"₹12,499.00",Cisco,CISCO SYSTEMS - ENTERPRISE,RV160W,28.96 x 25.4 x 10.67 cm; 1.17 Kilograms,RV160W,Dos,Cisco RV160W Wireless-AC VPN Router,1,No,802.11ac,4,"Wi-Fi, Ethernet",CISCO SYSTEMS - ENTERPRISE,China,1 kg 170 g
1,TP-link N300 WiFi Wireless Router TL-WR845N | ...,₹999.00,TP-Link,"TP-Link Technologies Co.,Ltd",TL-WR845N,NaN,TL-WR845N,Windows,"Wireless N Rotuer TL-WR845N, Power Supply Unit...",1,No,802.11b,5,Wireless,"TP-Link Technologies Co.,Ltd",China,281 g
2,"TP-Link AC750 Dual Band Wireless Cable Router,...","₹1,399.00",TP-Link,"TP-Link Technologies Co.,Ltd",NaN,NaN,ARCHER C20,Windows,NaN,NaN,NaN,5.8 GHz Radio Frequency,NaN,NaN,"TP-Link Technologies Co.,Ltd",China,320 g
3,TP-Link Archer AC1200 Archer C6 Wi-Fi Speed Up...,"₹2,399.00",TP-Link,"TP-Link Technologies Co.,Ltd",NaN,NaN,ARCHER C6,"Linux, Windows, Mac OS, Windows 10",NaN,NaN,NaN,802.11ac,NaN,NaN,"TP-Link Technologies Co.,Ltd",China,350 g
4,D-Link DIR-650IN Wireless N300 Router with 4 A...,₹899.00,D-Link,"D-Link Systems, Inc.",NaN,NaN,DIR-650IN,Windows,NaN,NaN,NaN,"802.11n, 802.11b, 802.11g",NaN,NaN,"D-Link Systems, Inc.",China,510 g
5,ASUS ZenWiFi AX Mini Mesh WiFi 6 Router (AX180...,"₹28,799.00",ASUS,Asus,XD4 (B-3-PK),NaN,XD4 (B-3-PK),NaN,"Node (XD4N), QSG, Warranty Card, Router (XD4R)...",3,No,"802.11n, 802.11ax, 802.11a, 802.11g, 802.11ac",2,Wi-Fi,Asus,China,295 g
6,Netgear WiFi Booster Range Extender - Covers u...,"₹2,899.00",Netgear,Netgear Technologies India PVT LTD. 21 Paharpu...,EX6120-100PES,NaN,EX6120-100PES,NaN,• AC1200 WiFi Range Extender (EX6120) • Quick ...,1,No,"802.11n, 802.11b, 802.11a, 802.11g, 802.11ac",1,NaN,Netgear Technologies India PVT LTD. 21 Paharpu...,Vietnam,130 g
7,D-Link DWA-131 Wireless N Nano USB Adapter (Bl...,₹499.00,D-Link,D-Link Corporation,DWA-131,NaN,DWA-131,NaN,"D-Link DWA-131 Wireless N Nano USB Adapter, Ma...",NaN,No,802.11n,NaN,NaN,D-Link Corporation,China,26.8 g
8,"TP-Link TD-W9970 300 Mbps Wireless VDSL/ADSL, ...","₹1,899.00",TP-Link,"TP-Link Technologies Co., Ltd",TD-W9970(UK),NaN,TD-W9970(UK),Windows,"TD-W9970, External Splitter, RJ-11 Telephone L...",NaN,No,802.11b,4,USB,"TP-Link Technologies Co., Ltd",China,300 g
9,D-Link DIR-825 MU-MIMO Gigabit Wireless Router...,"₹1,799.00",D-Link,D-Link Corporation,D-Link DIR-825,NaN,D-Link DIR-825,Windows,1 pc Router Power Adapter Ethernet Cable Wi-Fi...,1,No,"802.11n, 802.11b, 802.11a, 802.11g, 802.11ac",5,Wan;usb;lan;wireless,D-Link Corporation,China,1 kg 40 g


In [8]:
product.isnull().sum()

Name                   0
Price                  2
Brand                  0
Manufacturer           0
Model                 13
Package Dimensions    25
Item model number      1
Operating System       6
Mounting Hardware     13
Number Of Items       17
Batteries Required    13
Wireless Type          1
Number of Ports       13
Connector Type        14
Manufacturer           0
Country of Origin      1
Item Weight            0
dtype: int64